## Hangman Logic

In [73]:
def hangman(secret_word: str, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """Play a hangman game with the secret word with the provided guesser"""

    # Initialize the game state
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is",
              ' '.join(mask), 'length', len(secret_word))

    # Loop through the game
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        # Print the guess
        if verbose:
            print('Guess is', guess)

        # Check if the guess is already guessed
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            # Add the guess to the guessed set if it is valid and not already guessed
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                # Print the mistake if the guess is invalid
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1

        # Check if the game is over and break
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes

    # Print the secret word if the game is lost
    if verbose:
        print('Out of guesses. The word was', secret_word)
    # Return the number of mistakes made
    return mistakes


def test_guesser(guesser, test):
    """Measure the performance of a guesser on a test set of words and return 
    the average number of mistakes"""

    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

## Preprocessing

In [74]:
import nltk
import numpy as np
from nltk.corpus import brown


def preprocess(corpus):
    """Preprocess and shuffle the corpus"""

    corpus = set([word.lower() for word in corpus])
    corpus = list([word for word in corpus if word.isalpha()])
    np.random.shuffle(corpus)
    return corpus


# Download the Brown corpus
nltk.download('brown')
np.random.seed(1)

# Preprocess and split the corpus
brown_corpus = preprocess(brown.words())
training_set = brown_corpus[1000:]
test_set = brown_corpus[:1000]
print("Number of word types in train =", len(training_set))
print("Number of word types in test =", len(test_set))

[nltk_data] Downloading package brown to /home/vscode/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Number of word types in train = 39234
Number of word types in test = 1000


In [75]:
def test(guesser, guesser_name):
    """Test the guesser on the test set and print the results"""

    random_word = np.random.choice(test_set)
    print("Guessing word =", random_word)
    print(f"Number of mistakes made by the {guesser_name} =",
          hangman(random_word, guesser, 26, False))

    result = test_guesser(guesser, test_set)
    print(f"\nTesting the {guesser_name} using every word in test set")
    print("Average number of incorrect guesses: ", result)

## Human Guesser

In [76]:
def human(mask, guessed, **kwargs):
    """Play a hangman game interactively with the user"""

    print('\nEnter your guess:')
    return input().lower().strip()


# Set `interactive` to `True` to play the game manually
interactive = False
# interactive = True

if interactive:
    hangman('whatever', human, 8, True)

## Baseline Random Guesser

In [77]:
def random_guesser(mask, guessed, **kwargs) -> str:
    """Choose a random letter that is not already guessed"""

    # Remove the guessed letters
    whole_alphabet = set('abcdefghijklmnopqrstuvwxyz')
    remaining_alphabet: set[str] = whole_alphabet - guessed

    # Choose a random letter
    guess: str = np.random.choice(list(remaining_alphabet))
    return guess


test(random_guesser, "random guesser")

Guessing word = lapels
Number of mistakes made by the random guesser = 21

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.477


## Unigram Guesser

In [78]:
from collections import Counter


# Count the frequency of each letter in the training set
unigram_counts = Counter()
for word in training_set:
    unigram_counts.update(word)


def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    """Return the letter with the highest frequency"""

    # Remove the guessed letters
    whole_alphabet = set('abcdefghijklmnopqrstuvwxyz')
    remaining_alphabet: set[str] = whole_alphabet - guessed

    # Choose the most frequent letter
    sorted_remaining_alphabet = sorted(
        unigram_counts.items(), key=lambda item: item[1], reverse=True
    )
    for letter, freq in sorted_remaining_alphabet:
        if letter in remaining_alphabet:
            return letter


test(unigram_guesser, "unigram guesser")

Guessing word = prune
Number of mistakes made by the unigram guesser = 10

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.214


## Unigram Length Guesser

In [79]:
from collections import defaultdict


# Count the frequency of each letter in the training set by word length
unigram_counts_by_length = defaultdict(Counter)
for word in training_set:
    word_len = len(word)
    if word_len not in unigram_counts_by_length.keys():
        unigram_counts_by_length[word_len] = Counter()
    unigram_counts_by_length[word_len].update(word)


def unigram_length_guesser(
    mask,
    guessed,
    unigram_counts_by_length=unigram_counts_by_length,
    unigram_counts=unigram_counts
):
    """Return the most frequent letter given the length of the word"""

    # Remove the guessed letters
    whole_alphabet = set('abcdefghijklmnopqrstuvwxyz')
    remaining_alphabet: set[str] = whole_alphabet - guessed

    # Retrieve unigram counts for the given mask length
    unigram_counts_for_length = unigram_counts_by_length.get(len(mask))

    if unigram_counts_for_length:
        # Choose the most frequent letter given the mask length
        sorted_remaining_alphabet = sorted(
            unigram_counts_for_length.items(),
            key=lambda item: item[1],
            reverse=True
        )
        for letter, freq in sorted_remaining_alphabet:
            if letter in remaining_alphabet:
                return letter

    # Use unigrams if the letter does not exist in `unigram_counts_by_length`
    sorted_remaining_alphabet = sorted(
        unigram_counts.items(),
        key=lambda item: item[1],
        reverse=True
    )
    for letter, freq in sorted_remaining_alphabet:
        if letter in remaining_alphabet:
            return letter


test(unigram_length_guesser, "unigram length guesser")

Guessing word = destructive
Number of mistakes made by the unigram length guesser = 11

Testing the unigram length guesser using every word in test set
Average number of incorrect guesses:  10.175


## Bigram Guesser

In [80]:
# Count the frequency of each letter in the training set by two-letter sequences
bigram_counts = defaultdict(Counter)
for word in training_set:
    word = '$' + word  # Add a start symbol
    for i in range(1, len(word)):
        bigram_counts[word[i-1]].update(word[i])


def bigram_guesser(
    mask,
    guessed,
    bigram_counts=bigram_counts,
    unigram_counts=unigram_counts,
):
    """Return the most frequent letter given the previous letter in the word"""

    # Remove the guessed letters
    whole_alphabet = set('abcdefghijklmnopqrstuvwxyz')
    remaining_alphabet: set[str] = whole_alphabet - guessed

    letter_freq = dict()
    for letter in remaining_alphabet:
        letter_freq[letter] = 0

    mask: list[str] = ['$'] + mask
    for i in range(1, len(mask)):
        if mask[i] == '_':
            if mask[i-1] != '_':
                for letter in remaining_alphabet:
                    letter_freq[letter] += bigram_counts.get(
                        mask[i-1], 0).get(letter, 0)
            else:
                sorted_remaining_alphabet = sorted(
                    unigram_counts.items(),
                    key=lambda item: item[1],
                    reverse=True
                )
                for letter, freq in sorted_remaining_alphabet:
                    if letter in remaining_alphabet:
                        return letter

    # Choose the letter with the highest sum of probabilities
    return max(letter_freq, key=letter_freq.get)


test(bigram_guesser, "bigram guesser")

Guessing word = twenty
Number of mistakes made by the bigram guesser = 10

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.743


## Advanced Guesser

In [81]:
# Bigrams and reverse bigrams
bigram_counts = defaultdict(Counter)
for word in training_set:
    word = '$' + word
    for i in range(1, len(word)):
        bigram_counts[word[i-1]].update(word[i])
rev_bigram_counts = defaultdict(Counter)
for word in training_set:
    word = word + '$'
    for i in range(0, len(word)-1):
        rev_bigram_counts[word[i+1]].update(word[i])

# Trigrams and reverse trigrams
trigram_counts = defaultdict(lambda: defaultdict(Counter))
for word in training_set:
    word = '$$' + word
    for i in range(2, len(word)):
        trigram_counts[word[i-2]][word[i-1]].update(word[i])
rev_trigram_counts = defaultdict(lambda: defaultdict(Counter))
for word in training_set:
    word = word + '$$'
    for i in range(0, len(word)-2):
        rev_trigram_counts[word[i+2]][word[i+1]].update(word[i])

# Bidirectional bigrams (trigrams)
bidir_bigram_counts = defaultdict(lambda: defaultdict(Counter))
for word in training_set:
    word = '$' + word + '$'
    for i in range(1, len(word)-1):
        bidir_bigram_counts[word[i-1]][word[i+1]].update([word[i]])

# Bidirectional trigrams (pentagrams)
bidir_trigram_counts = defaultdict(
    lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(Counter)))
)
for word in training_set:
    word = '$$' + word + '$$'
    for i in range(2, len(word)-2):
        bidir_trigram_counts[word[i-2]][word[i-1]
                                        ][word[i+1]][word[i+2]].update([word[i]])


def advanced_guesser(
    mask,
    guessed,
    unigram_counts=unigram_counts,
    bigram_counts=bigram_counts,
    rev_bigram_counts=rev_bigram_counts,
    trigram_counts=trigram_counts,
    rev_trigram_counts=rev_trigram_counts,
    bidir_bigram_counts=bidir_bigram_counts,
    bidir_trigram_counts=bidir_trigram_counts,
) -> str:
    """Return the most frequent letter given bigram, trigram, and pentagram frequencies"""

    # Remove the guessed letters
    whole_alphabet = set('abcdefghijklmnopqrstuvwxyz')
    remaining_alphabet: set[str] = whole_alphabet - guessed

    ngram_count = dict()
    for letter in remaining_alphabet:
        ngram_count[letter] = 0

    mask = ['$', '$', '$'] + mask + ['$', '$', '$']
    for i in range(3, len(mask)-3):
        if mask[i] == '_':
            # Pentagram
            if mask[i-2] != '_' and mask[i-1] != '_' and mask[i+1] != '_' and mask[i+2] != '_':
                for letter in remaining_alphabet:
                    ngram_count[letter] += bidir_trigram_counts.get(mask[i-2], {}).get(
                        mask[i-1], {}).get(mask[i+1], {}).get(mask[i+2], {}).get(letter, 0)

            # Trigram
            elif mask[i-1] != '_' and mask[i+1] != '_':
                for letter in remaining_alphabet:
                    ngram_count[letter] += bidir_bigram_counts.get(
                        mask[i-1], {}).get(mask[i+1], {}).get(letter, 0)
            elif mask[i-2] != '_' and mask[i-1] != '_':
                for letter in remaining_alphabet:
                    ngram_count[letter] += trigram_counts.get(
                        mask[i-2], {}).get(mask[i-1], {}).get(letter, 0)
            elif mask[i+2] != '_' and mask[i+1] != '_':
                for letter in remaining_alphabet:
                    ngram_count[letter] += rev_trigram_counts.get(
                        mask[i+2], {}).get(mask[i+1], {}).get(letter, 0)

            # Bigram
            elif mask[i-1] != '_':
                for letter in remaining_alphabet:
                    ngram_count[letter] += bigram_counts.get(
                        mask[i-1], {}).get(letter, 0)
            elif mask[i+1] != '_':
                for letter in remaining_alphabet:
                    ngram_count[letter] += rev_bigram_counts.get(
                        mask[i+1], {}).get(letter, 0)

            # Unigram
            else:
                for letter in remaining_alphabet:
                    ngram_count[letter] += unigram_counts.get(letter, 0)
    return max(ngram_count, key=ngram_count.get)


test(advanced_guesser, "advanced guesser")

Guessing word = curriculums
Number of mistakes made by the advanced guesser = 7

Testing the advanced guesser using every word in test set
Average number of incorrect guesses:  7.34
